# Football Analysis (2012 to 2023) Data engineering project 

#### GET Postgres to work on Python

In [23]:
import psycopg2 #Postgres with Python


#### Establishing the connection with Postgres and Creating a new Database

In [80]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=Hammaad@Rizwan")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)#Establishing conneion with the database
    
try: 
    cur = conn.cursor()#Cursor is a method tat allows us to use the database.Allows Python code to execute PostgreSQL command in a database session
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

conn.set_session(autocommit=True)#like autosave
try: 
    cur.execute("create database football_Analysis")#Creates a table called football analysis
except psycopg2.Error as e:
    print(e)

database "football_analysis" already exists



#### Closes the main connection, and reopens it in the new Database

In [57]:
try: 
    conn.close()#Closes the connection
except psycopg2.Error as e:
    print(e)
    
try: #Reopens the connection under the new database
    conn = psycopg2.connect("host=127.0.0.1 dbname=football_analysis user=postgres password=Hammaad@Rizwan")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

#### Creating 4 tables


4 Tables have been created with the relavant Primary and forieng keys 

In [41]:
cur.execute("CREATE TABLE competitions(competition_ID VARCHAR, Name VARCHAR, type VARCHAR,sub_type VARCHAR,PRIMARY KEY(competition_ID));")
cur.execute("CREATE TABLE clubs(club_ID int,Name VARCHAR,competition_ID VARCHAR,squad_size int,staduim_name VARCHAR, National_team_player int, Coach_Name VARCHAR,PRIMARY KEY(club_ID),Foreign KEY(competition_ID) REFERENCES competitions(competition_ID));")
cur.execute("CREATE TABLE players(player_ID int,club_ID int,Date_of_birth VARCHAR,sub_position VARCHAR, foot VARCHAR, height int,competition_ID VARCHAR,First_name VARCHAR,Last_name VARCHAR,PRIMARY KEY(player_ID),Foreign KEY(club_ID) REFERENCES clubs(club_ID),Foreign KEY(competition_ID) REFERENCES competitions(competition_ID));")
cur.execute("CREATE TABLE GAMES(Game_ID int,competition_ID VARCHAR, Date VARCHAR, Home_club_ID int,Away_club_ID int,Home_club_goals int,Away_club_goals int,Foreign KEY(Home_club_ID) REFERENCES clubs(club_ID),Foreign KEY(Away_club_ID) REFERENCES clubs(club_ID),Foreign KEY(competition_ID) REFERENCES competitions(competition_ID));")


DuplicateTable: relation "competitions" already exists


#### Popuating the tables from the CSV files using Panda

In [42]:
import pandas as pd

In [116]:
competitions = pd.read_csv("competitons.csv")
clubs = pd.read_csv("C:\\Users\\Hammaad\\Documents\\clubs.csv")
players = pd.read_csv("C:\\Users\\Hammaad\\Documents\\players.csv")
games = pd.read_csv("game.csv")

Reading the csv file using pandas and then by iterating each row we insert it to the relavant table

In [127]:
players.head()

,player_id,current_club_id,date_of_birth,sub_position,foot,height_in_cm,current_club_domestic_competition_id,first_name,last_name
0,134354.0,498.0,30 01 1995,Centre-Forward,Right,0.0,BE1,Ian,Raeymaekers
1,99946.0,1095.0,20 09 1990,Centre-Forward,Right,180.0,FR1,Mohamed,Camara
2,76948.0,979.0,8 12 1987,Centre-Forward,Right,175.0,PO1,Pablo,Olivera
3,108372.0,38.0,6 02 1992,Centre-Forward,Right,178.0,L1,Aliosman,Aydin
4,78820.0,354.0,3 01 1984,Centre-Forward,Right,184.0,BE1,Jaime Alfonso,Ruiz


In [123]:
clubs 

,club_id,name,domestic_competition_id,squad_size
0,3,1 Fc Koln,L1,31
1,4,1 Fc Nurnberg,L1,28
2,5,Ac Mailand,IT1,31
3,6,Adanaspor,TR1,27
4,10,Arminia Bielefeld,L1,25
...,...,...,...,...
406,60551,Sk Dnipro 1,UKR1,27
407,60949,Volos Nps,GR1,29
408,61825,Fk Minaj,UKR1,25
409,68608,Cf Os Belenenses,PO1,28


In [143]:
try:
    # Establish a connection to your PostgreSQL database
    conn = psycopg2.connect("host=127.0.0.1 dbname=football_analysis user=postgres password=Hammaad@Rizwan")
    cur = conn.cursor()

    # Begin a new transaction
    cur.execute("BEGIN")

    # Define your SQL statement for inserting data into the 'competitions' table
    
    players_insert = ("""INSERT INTO players(
    player_ID ,
    club_ID ,
    Date_of_birth ,
    sub_position ,
    foot ,
    height ,
    competition_ID ,
    First_name ,
    Last_name)  VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)""")
    #games_insert = ("""INSERT INTO games(Game_ID,competition_ID , Date , Home_club_ID ,Away_club_ID ,Home_club_goals ,Away_club_goals ) VALUES(%s,%s,%s,%s,%s,%s,%s)""")
    
    
    # Iterate over the rows of the 'competitions' DataFrame and execute the SQL statement
 
    for i, row in players.iterrows():
        ListOfPlayers=list(row)
        for j in range (len(ListOfPlayers)):
            if ListOfPlayers[2]=="":
                ListOfPlayers[2]="None"
            if ListOfPlayers[3]=="":
                ListOfPlayers[3]="None"
            if ListOfPlayers[4]=="":
                ListOfPlayers[4]="None"
            if ListOfPlayers[5]=="":
                ListOfPlayers[5]=0
            if ListOfPlayers[6]=="":
                ListOfPlayers[6]="None"
            if ListOfPlayers[7]=="":
                ListOfPlayers[7]="None"
            if ListOfPlayers[8]=="":
                ListOfPlayers[8]="None"
        cur.execute(players_insert, ListOfPlayers)
    cur.execute("COMMIT")

except psycopg2.DatabaseError as e:
    # Rollback the transaction and handle the error
    conn.rollback()
    print("Transaction failed and rolled back:", str(e))

finally:
    # Close the cursor and connection
    if cur:
        cur.close()
    if conn:
        conn.close()


Transaction failed and rolled back: integer out of range



In [ ]:
nan_values = players.isna()
count=0
for i in range(len(nan_values)):
    nan_values[i]
#for i, row in players.iterrows():
    

In [148]:
try:
    # Establish a connection to your PostgreSQL database
    conn = psycopg2.connect("host=127.0.0.1 dbname=football_analysis user=postgres password=Hammaad@Rizwan")
    cur = conn.cursor()

    # Begin a new transaction
    cur.execute("BEGIN")
    # Define your SQL statement for inserting data into the 'competitions' table
    
    # Iterate over the rows of the 'competitins' DataFrame and execute the SQL statement
    """for i, row in clubs.iterrows():
        cur.execute(clubs_insert, list(row))
    # Commit the transaction
    cur.execute("COMMIT")
    print("Transaction committed successfully!")"""

except psycopg2.DatabaseError as e:
    # Rollback the transaction and handle the error
    conn.rollback()
    print("Transaction failed and rolled back:", str(e))



In [155]:
players.index

RangeIndex(start=0, stop=59933, step=1)